# Setup

In [57]:
import re
import nltk

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torchtext.vocab import build_vocab_from_iterator


import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.preprocessing import LabelEncoder
nltk.download('punkt')

from tqdm.auto import tqdm

RANDOM_SEED = 42


# Set the seed value all over the place to make this reproducible.
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed_all(RANDOM_SEED)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


device(type='cpu')

# Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
cd drive/MyDrive/datasets

[Errno 2] No such file or directory: 'drive/MyDrive/datasets'
/content


# Функция обучения

In [82]:
def train(model, optimizer, criterion, n_epochs, train_loader, test_loader):

  loss_train = []
  accuracy_train = []

  model.to(device)

  for epoch in range(n_epochs):
    model.train()
    for batch in tqdm(train_loader, desc=f"Training epoch {epoch + 1}/{n_epochs}"):
        inputs = batch["X"].to(device)
        labels = batch["y"].to(device)
        output = model(inputs)

        loss = criterion(output, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
      for batch in tqdm(test_loader, desc=f"Testing epoch {epoch + 1}/{n_epochs}"):
        inputs = batch["X"].to(device)
        labels = batch["y"].to(device)
        output = model(inputs)
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    test_accuracy = correct / total
    accuracy_train.append(test_accuracy)
    print('Epoch [{}/{}], Loss: {:.4f}, Test Accuracy: {:.2f}%'.format(epoch + 1, n_epochs, loss.item(), test_accuracy * 100))
    loss_train.append(loss.item())


## 1. Генерирование русских имен при помощи RNN

Датасет: https://disk.yandex.ru/i/2yt18jHUgVEoIw

1.1 На основе файла name_rus.txt создайте датасет.
  * Учтите, что имена могут иметь различную длину
  * Добавьте 4 специальных токена:
    * `<PAD>` для дополнения последовательности до нужной длины;
    * `<UNK>` для корректной обработки ранее не встречавшихся токенов;
    * `<SOS>` для обозначения начала последовательности;
    * `<EOS>` для обозначения конца последовательности.
  * Преобразовывайте строку в последовательность индексов с учетом следующих замечаний:
    * в начало последовательности добавьте токен `<SOS>`;
    * в конец последовательности добавьте токен `<EOS>` и, при необходимости, несколько токенов `<PAD>`;
  * `Dataset.__get_item__` возращает две последовательности: последовательность для обучения и правильный ответ.
  
  Пример:
  ```
  s = 'The cat sat on the mat'
  # преобразуем в индексы
  s_idx = [2, 5, 1, 2, 8, 4, 7, 3, 0, 0]
  # получаем x и y (__getitem__)
  x = [2, 5, 1, 2, 8, 4, 7, 3, 0]
  y = [5, 1, 2, 8, 4, 7, 3, 0, 0]
  ```

### Read file

In [12]:
with open('/content/drive/MyDrive/DL_FU/name_rus.txt', 'r',  encoding = 'cp1251') as file:
  lines = file.readlines()

In [14]:
lines = [line.replace('\n', '' ) for line in lines]

In [17]:
lines

'яша'

### Vocab

In [21]:
vocab = build_vocab_from_iterator(lines, specials=["<PAD>", "<UNK>", "<SOS>", "<EOS>"])
token2inx = vocab.get_stoi()
inx2token = {inx: token for token, inx in vocab.get_stoi().items()}

In [22]:
token2inx

{'ж': 32,
 'э': 31,
 'з': 30,
 'б': 29,
 'к': 11,
 'е': 9,
 'о': 14,
 'н': 5,
 'а': 4,
 'в': 16,
 'ы': 28,
 'и': 6,
 'х': 21,
 '<SOS>': 2,
 '<EOS>': 3,
 '<UNK>': 1,
 'я': 8,
 'л': 7,
 '<PAD>': 0,
 'ш': 19,
 'с': 13,
 'м': 15,
 'ю': 17,
 'у': 18,
 'д': 20,
 'г': 22,
 'т': 12,
 'п': 23,
 'ч': 24,
 'й': 26,
 'ц': 33,
 'ь': 25,
 'р': 10,
 'ф': 27}

### X и y

#### Sample

In [41]:
# maxlen = 40
# step = 3
# sentences = []
# next_chars = []
# for i in range(0, len(text_nietzsche) - maxlen, step):
#     sentences.append(text_nietzsche[i: i + maxlen])
#     next_chars.append(text_nietzsche[i + maxlen])
# print('nb sequences:', len(sentences))

# print('Vectorization...')
# x = np.zeros((len(sentences), maxlen, len(token2inx)))
# y = np.zeros((len(sentences), len(token2inx)))
# for i, sentence in enumerate(sentences):
#     for t, char in enumerate(sentence):
#         x[i, t, token2inx[char]] = 1
#     y[i, token2inx[next_chars[i]]] = 1


maxlen = 3
step = 1
X = []
y = []

for name in lines:
  print(name)
  name1 = [token2inx.get(char,1) for char in name]
  name1.insert(0, token2inx["<SOS>"])
  name1.append(token2inx["<EOS>"])
  for i in range(0, len(name) - maxlen, step):
    print(name[i: i + maxlen], name1[i: i + maxlen])
    print(name[i + maxlen], name1[i + maxlen])
    X.append(name1[i: i + maxlen])
    y.append(name1[i + maxlen])
  break


авдокея
авд [2, 4, 16]
о 20
вдо [4, 16, 20]
к 14
док [16, 20, 14]
е 11
оке [20, 14, 11]
я 9


#### Vector

In [45]:
maxlen = 3
step = 1
X = []
y = []

for name in lines:
  name = [token2inx.get(char,1) for char in name]
  name.insert(0, token2inx["<SOS>"])
  name.append(token2inx["<EOS>"])
  for i in range(0, len(name) - maxlen, step):
    X.append(name[i: i + maxlen])
    y.append(name[i + maxlen])

X = np.array(X)
y = np.array(y)

In [46]:
X

array([[ 2,  4, 16],
       [ 4, 16, 20],
       [16, 20, 14],
       ...,
       [16, 11,  4],
       [ 2,  8, 19],
       [ 8, 19,  4]])

In [47]:
y

array([20, 14, 11, ...,  3,  4,  3])

### Train and test

In [64]:
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = RANDOM_SEED)

### class Dataset

In [91]:
class NameDataset(Dataset):
  def __init__(self, X, y):
    self.X = X
    self.y = y


  def __len__(self):
    return len(self.X)

  def __getitem__(self, item):
    return {"X" : torch.tensor(self.X[item]).to(torch.long) ,
            "y" : torch.tensor(self.y[item]).to(torch.long)}

In [92]:
train_dataset = NameDataset(X_train, y_train)
test_dataset = NameDataset(X_test, y_test)

### class DataLoader

In [93]:
BATCH_SIZE = 64
train_data_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE )
test_data_loader = DataLoader(test_dataset, batch_size = BATCH_SIZE)

1.2 Создайте и обучите модель для генерации фамилии.

  * Для преобразования последовательности индексов в последовательность векторов используйте `nn.Embedding`;
  * Используйте рекуррентные слои;
  * Задача ставится как предсказание следующего токена в каждом примере из пакета для каждого момента времени. Т.е. в данный момент времени по текущей подстроке предсказывает следующий символ для данной строки (задача классификации);
  * Примерная схема реализации метода `forward`:
  ```
    input_X: [batch_size x seq_len] -> nn.Embedding -> emb_X: [batch_size x seq_len x embedding_size]
    emb_X: [batch_size x seq_len x embedding_size] -> nn.RNN -> output: [batch_size x seq_len x hidden_size]
    output: [batch_size x seq_len x hidden_size] -> torch.Tensor.reshape -> output: [batch_size * seq_len x hidden_size]
    output: [batch_size * seq_len x hidden_size] -> nn.Linear -> output: [batch_size * seq_len x vocab_size]
  ```


In [141]:
class RNNGeneratorNames(nn.Module):
  def __init__(self, vocab_size, embedding_dim, hidden_size):
      super().__init__()

      self.embedding = nn.Embedding(vocab_size, embedding_dim)
      self.rnn = nn.RNN(embedding_dim, hidden_size, batch_first=True)
      self.fc = nn.Linear(hidden_size,1)

  def forward(self, input_X):
    emb_X = self.embedding(input_X)
    output, _ = self.rnn(emb_X)

    batch_size = output.shape[0]
    seq_len = output.shape[1]
    output = output.reshape(batch_size * seq_len, hidden_size)

    output = self.fc(output)
    return output

In [142]:
vocab_size = len(token2inx)
embedding_dim = 6
hidden_size = 100
EPOCHS = 10


modelNameGeneration = RNNGeneratorNames(vocab_size, embedding_dim, hidden_size)


optimizer = optim.Adam(modelNameGeneration.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [143]:
train(modelNameGeneration, optimizer, criterion, EPOCHS, train_data_loader, test_data_loader)

Training epoch 1/10:   0%|          | 0/128 [00:00<?, ?it/s]

torch.Size([192, 1])


ValueError: ignored

1.3 Напишите функцию, которая генерирует фамилию при помощи обученной модели:
  * Построение начинается с последовательности единичной длины, состоящей из индекса токена `<SOS>`;
  * Начальное скрытое состояние RNN `h_t = None`;
  * В результате прогона последнего токена из построенной последовательности через модель получаете новое скрытое состояние `h_t` и распределение над всеми токенами из словаря;
  * Выбираете 1 токен пропорционально вероятности и добавляете его в последовательность (можно воспользоваться `torch.multinomial`);
  * Повторяете эти действия до тех пор, пока не сгенерирован токен `<EOS>` или не превышена максимальная длина последовательности.

При обучении каждые `k` эпох генерируйте несколько фамилий и выводите их на экран.

## 2. Генерирование текста при помощи RNN

2.1 Скачайте из интернета какое-нибудь художественное произведение
  * Выбирайте достаточно крупное произведение, чтобы модель лучше обучалась;

2.2 На основе выбранного произведения создайте датасет.

Отличия от задачи 1:
  * Токены <SOS>, `<EOS>` и `<UNK>` можно не добавлять;
  * При создании датасета текст необходимо предварительно разбить на части. Выберите желаемую длину последовательности `seq_len` и разбейте текст на построки длины `seq_len` (можно без перекрытия, можно с небольшим перекрытием).

2.3 Создайте и обучите модель для генерации текста
  * Задача ставится точно так же как в 1.2;
  * При необходимости можете применить:
    * двухуровневые рекуррентные слои (`num_layers`=2)
    * [обрезку градиентов](https://pytorch.org/docs/stable/generated/torch.nn.utils.clip_grad_norm_.html)

2.4 Напишите функцию, которая генерирует фрагмент текста при помощи обученной модели
  * Процесс генерации начинается с небольшого фрагмента текста `prime`, выбранного вами (1-2 слова)
  * Сначала вы пропускаете через модель токены из `prime` и генерируете на их основе скрытое состояние рекуррентного слоя `h_t`;
  * После этого вы генерируете строку нужной длины аналогично 1.3


2.1 Скачайте из интернета какое-нибудь художественное произведение
  * Выбирайте достаточно крупное произведение, чтобы модель лучше обучалась;

2.2 На основе выбранного произведения создайте датасет.

Отличия от задачи 1:
  * Токены <SOS>, `<EOS>` и `<UNK>` можно не добавлять;
  * При создании датасета текст необходимо предварительно разбить на части. Выберите желаемую длину последовательности `seq_len` и разбейте текст на построки длины `seq_len` (можно без перекрытия, можно с небольшим перекрытием).